In [ ]:
!rm -rf Dataset Trained_Model                   # Remove existing directory
!pip install -U gdown --pre >/dev/null          # Install gdown to download file from GDrive
!gdown 1-4Z9DcqgR2IOqiqd27Lwlc_GYtVPwoq0        # Download FCN32 Model
!gdown 10KV85__YEcXbCJQKDKslyB4eMjhU68KA        # Download FCN8 Model
!gdown 1u4WJLjYrbZHwdvFOHQXJqDTtco6F5hJ-        # Download dataset from GDrive by file ID
!unzip -q Dataset.zip; rm Dataset.zip           # Extract the dataset zip file

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras import backend as K
import os
import numpy as np
from random import sample
from skimage.io import imread
from matplotlib import pyplot as plt

def iou_coef(y_true, y_pred, smooth=1):
    intersection = K.sum(K.abs(y_true * y_pred), axis=[1,2,3])
    union = K.sum(y_true,[1,2,3])+K.sum(y_pred,[1,2,3])-intersection
    iou = K.mean((intersection + smooth) / (union + smooth), axis=0)
    return iou

def dice_coef(y_true, y_pred, smooth = 1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def soft_dice_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)

model32 = load_model("FCNN-32.h5", custom_objects={'soft_dice_loss': soft_dice_loss, 'iou_coef': iou_coef})
model8 = load_model("Model-FCN-8.h5", custom_objects={'soft_dice_loss': soft_dice_loss, 'iou_coef': iou_coef})






import os
import cv2
import numpy as np
from random import sample
from skimage.io import imread
from sklearn.model_selection import train_test_split


IMAGES_PATH = 'Dataset/Images/'
MASKS_PATH  = 'Dataset/Masks/'
TEST_PATH   = 'Dataset/Test_Images/'

# Number of images to use (Larger the number, more RAM required)
N_IMAGES = 500

# Imread each image and save to an array

sat_imgs = os.listdir(IMAGES_PATH)
msk_imgs = os.listdir(MASKS_PATH)
sat_imgs.sort(), msk_imgs.sort()

images = []
for image in sat_imgs[3000:3000+N_IMAGES]:
    data = imread(IMAGES_PATH + image)
    images.append(data)

masks = []
for mask in msk_imgs[3000:3000+N_IMAGES]:
    data = imread(MASKS_PATH + mask)
    data = cv2.cvtColor(data, cv2.COLOR_BGR2GRAY)
    data = np.expand_dims(data, axis=-1)
    masks.append(data)

images = np.stack(images)
masks = np.stack(masks) / 255

train_images, test_images, train_masks, test_masks = train_test_split(images, masks, test_size=0.3, random_state=2)

In [ ]:
IMAGES_PATH = 'Dataset/Images/'

pred_images = sample(os.listdir(IMAGES_PATH), 4)
pred_masks  = []
for mask_name in pred_images:
    mask_name = mask_name.replace('_sat.jpg', '_mask.png')
    pred_masks.append(mask_name)

In [ ]:
f = plt.figure(figsize = (15, 12), constrained_layout=True)
gs = f.add_gridspec(5, 6)
titles = ['Input Image', 'Ground Truth', 'Predicted Image']

for i in range(0,4):
  data = [imread(f'Dataset/Images/{pred_images[i]}')]
  input_img = imread(f'Dataset/Images/{pred_images[i]}')
  mask_img = imread(f'Dataset/Masks/{pred_masks[i]}')
  test_data = np.asarray([input_img])
  output = model8.predict(test_data, verbose=0)[0][:,:,0]
  final_out = input_img
  for xi in range(len(output)):
    for yi in range(len(output[xi])):
      if output[xi][yi] > 0.1:
        final_out[xi][yi] = [255, 255, 0]
  data += [mask_img, final_out]
  for j in range(0,3):  
    f.add_subplot(gs[i, j])
    plt.imshow(data[j])
    plt.axis('off')
    if i == 0:
      plt.title(titles[j])
plt.show()

In [ ]:
f = plt.figure(figsize = (15, 12), constrained_layout=True)
gs = f.add_gridspec(5, 6)
titles = ['Input Image', 'Ground Truth', 'Predicted Image']

for i in range(0,4):
  data = [imread(f'Dataset/Images/{pred_images[i]}')]
  input_img = imread(f'Dataset/Images/{pred_images[i]}')
  mask_img = imread(f'Dataset/Masks/{pred_masks[i]}')
  test_data = np.asarray([input_img])
  output = model32.predict(test_data, verbose=0)[0][:,:,0]
  final_out = input_img
  for xi in range(len(output)):
    for yi in range(len(output[xi])):
      if output[xi][yi] > 0.1:
        final_out[xi][yi] = [255, 255, 0]
  data += [mask_img, final_out]
  for j in range(0,3):  
    f.add_subplot(gs[i, j])
    plt.imshow(data[j])
    plt.axis('off')
    if i == 0:
      plt.title(titles[j])
plt.show()

In [ ]:
model32.evaluate(test_images, test_masks)

In [ ]:
model8.evaluate(test_images, test_masks)